# Dataset creator Notebook

>This notebook downloads specific classes from the Open Images dataset with the desired cardinality. Then, it transform this set into .tfrecord format to make it possible to use with Tensorflow Object Detection API.
>
>The downloaded set can be saved into the mounted Drive folder.

## 1. Prepare the environment

### Clone MISC repos with Git

In [2]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 422, done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 422
Receiving objects: 100% (422/422), 34.08 MiB | 33.30 MiB/s, done.
Resolving deltas: 100% (146/146), done.


### Install Object Detection module

In [3]:
!pip3 install tensorflow-object-detection-api

     |████████████████████████████████| 583kB 2.7MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 2.6MB 12.7MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp36-none-any.whl size=844515 sha256=1b8fe322ad80e98470401558aebca291759151d43d9b58661eee20d25598660c
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
Successfully built tensorflow-object-detection-api


### Import required dependencies

In [4]:
import imageio
import zipfile
import shutil
import os
import csv
import tensorflow as tf
import pandas as pd
import contextlib2
import gc
from PIL import Image
from google.colab import drive
from pathlib import Path
from datetime import datetime
from object_detection.utils import dataset_util

### Define paths

In [71]:
rootPath = '/content/drive/My Drive/Machine Learning/Stolen vehicle detection'
dataPath = rootPath + '/data'

localPath = '/content'
rootOIDv4Path = localPath + '/OIDv4_ToolKit'
datasetRootPath = rootOIDv4Path + '/OID'
generatedDatasetPath = localPath + '/generated'
recordsPath = generatedDatasetPath + '/records'
csvPath = generatedDatasetPath + '/csv'

### Install OIDv4 requirements

In [6]:
os.chdir(rootOIDv4Path)
!pip3 install -r requirements.txt

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 6.7MB 12.8MB/s 
     |████████████████████████████████| 552kB 40.0MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for awscli: filename=awscli-1.18.153-py2.py3-none-any.whl size=3354722 sha256=d6e9e9985461907c1ecb11bf8c97832054c8cd18814ca019ca3872b63746751e
  Stored in directory: /root/.cache/pip/wheels/2d/42/20/ce50d9438e1ce7655a3c2b928221afb57749bd140c74d70845
Successfully built awscli
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6


### Mount Google Drive to this Notebook instance
>As the dataset has been prepared previously and updated to Google Drive, the model building and training process will be done there, not locally.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir(localPath)
# Show current directory
!pwd

Mounted at /content/drive
/content


##2. Get the data
>There are 3 possibilities to get the data.

#### A: extract a previously downloaded raw dataset from Drive

In [8]:
# Run it only when the dataset is not yet extracted
'''
zipRef = zipfile.ZipFile(dataPath + "/dataset.zip", 'r')
zipRef.extractall(localPath + "/data")
zipRef.close()
print('Dataset extraction successful')
'''

Dataset extraction successful


#### B: extract a previously downloaded raw dataset placed under `/content`

In [ ]:
# Run it only when the dataset is not yet extracted 
'''
zipRef = zipfile.ZipFile("/content/dataset.zip", 'r')
zipRef.extractall(localPath + "/data")
zipRef.close()
print('Dataset extraction successful')
'''

Dataset extraction successful


#### C: download from Open Image Dataset

##### show available OIDv4 commands

In [7]:
os.chdir(rootOIDv4Path)
!python3 main.py -h

usage: main.py [-h] [--Dataset /path/to/OID/csv/] [-y]
               [--classes list of classes [list of classes ...]]
               [--type_csv 'train' or 'validation' or 'test' or 'all']
               [--sub Subset of human verified images or machine generated h or m)]
               [--image_IsOccluded 1 or 0] [--image_IsTruncated 1 or 0]
               [--image_IsGroupOf 1 or 0] [--image_IsDepiction 1 or 0]
               [--image_IsInside 1 or 0] [--multiclasses 0 (default or 1]
               [--n_threads [default 20]] [--noLabels]
               [--limit integer number]
               <command> 'downloader', 'visualizer' or 'ill_downloader'.

Open Image Dataset Downloader

positional arguments:
  <command> 'downloader', 'visualizer' or 'ill_downloader'.
                        'downloader', 'visualizer' or 'ill_downloader'.

optional arguments:
  -h, --help            show this help message and exit
  --Dataset /path/to/OID/csv/
                        Directory of the OID da

##### Download train, test, validation sets

In [ ]:
!python3 main.py downloader --classes "Vehicle registration plate" --type_csv train --multiclasses 0 --limit 5368


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Vehicle registration plate.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 30461 KB/s, 0 seconds passed

In [ ]:
!python3 main.py downloader --classes "Vehicle registration plate" --type_csv test --multiclasses 0 --limit 1113


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Vehicle registration plate.
   [ERROR] | Missing the test-annotations-bbox.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...100%, 49 MB, 25649 KB/s, 1 seconds passed
[DO

In [ ]:
!python3 main.py downloader --classes "Vehicle registration plate" --type_csv validation --multiclasses 0 --limit 386


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Vehicle registration plate.
   [ERROR] | Missing the validation-annotations-bbox.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...100%, 16 MB, 11274 KB/s, 1 seconds pass

##### Save the downloaded set to Drive

In [ ]:
shutil.make_archive("/content/dataset", 'zip', datasetRootPath)
shutil.move("/content/dataset.zip", dataPath + "/dataset.zip")
print(f'Dataset successfully zipped & moved to: {dataPath}')

Dataset successfully zipped & moved to: /content/drive/My Drive/Machine Learning/Stolen vehicle detection/data


##3. Create supplementary files
>Create a `txt` file containing the class names, and `tfrecord`s containing the images and their annotations.

### Define class names to process
>Order of the classes counts!

In [74]:
classNames = ["Vehicle registration plate", "Vehicle"]

### Create class names file

#### Create class names `txt` file

In [75]:
def createClassTxt(destinationDir, classNames):

  os.chdir(destinationDir)
  # creating the `classes.txt` file
  classesPath = os.path.join("classes.txt")

  content = ""

  # creates a txt file that contains the class names
  for i, className in enumerate(classNames):

      content = (
          content + 
          f"{className}\n"
      )

  content = content.strip()

  with open(classesPath, "w") as f:
      f.write(content)

  print("Class names txt file creation successful. Destination: %s" %(destinationDir + '/' + classesPath))

#### Create class names `pbtxt` file

In [76]:
def createClassPbtxt(destinationDir, classNames):

  os.chdir(destinationDir)
  # creating the `classes.pbtxt` file
  classesPath = os.path.join("classes.pbtxt")

  content = ""

  # creates a pbtxt file that contains the class names
  for i, className in enumerate(classNames):

      content = (
          content + 
          f"item {{\n    id: {i + 1}\n    name: '{className}'\n }}\n\n"
      )

  content = content.strip()

  with open(classesPath, "w") as f:
      f.write(content)

  print("Class names pbtxt file creation successful. Destination: %s" %(destinationDir + '/' + classesPath))

### Create the `tfrecord` files
>These files are the inputs of the Tensorflow Object Detection API models. 

#### Create one `tfrecord` from the input parameters

In [77]:
'''
Creates the tfrecord element from the input.

  Args:
    path: path of the image
    imageId: Id of the current image to create records for
    mustPresentClasses: list of class names that must present (returns empty record otherwise)
    classes: dict of classes that is added to the record if present
    imageAnnotations: dataframe containing the image BBs

  Returns:
    TFRecord
'''
def createRecord(path, imageId, mustPresentClasses, classes, imageAnnotations):

  image = Image.open(path)
  imgWidth, imgHeight = image.size
  imgData = tf.io.gfile.GFile(path, 'rb').read()

  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classesText = []
  classesInt = []

  for mustClass in mustPresentClasses:

    if(mustClass not in imageAnnotations['LabelName'].values):
      return None

  for _, row in imageAnnotations.loc[imageAnnotations['LabelName'].isin(classes.keys())].iterrows():

      xmins.append(row['XMin'])
      xmaxs.append(row['XMax'])
      ymins.append(row['YMin'])
      ymaxs.append(row['YMax'])
      classesText.append(row['LabelName'].encode('utf8'))
      classesInt.append(classes[row['LabelName']])

  tFRecord = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(imgHeight),
      'image/width': dataset_util.int64_feature(imgWidth),
      'image/filename': dataset_util.bytes_feature(imageId.encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(imageId.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(imgData),
      'image/format': dataset_util.bytes_feature(b'jpg'),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classesText),
      'image/object/class/label': dataset_util.int64_list_feature(classesInt)
  }))

  return tFRecord

#### Create a `csv` row from the input parameters

##### Encode list elements as a row

In [78]:
def appendListAsRow(fileName, list):

    # Open file in append mode
    with open(fileName, 'a+', newline='') as writeObj:

      # Create a writer object from csv module
      writer = csv.writer(writeObj)

      # Add contents of list as last row in the csv file
      writer.writerow(list)

##### Create `csv` rows of an image

In [79]:
def createCsvRowsOfImage(imgPath, imageId, classes, annotations):

  image = Image.open(imgPath)
  imgWidth, imgHeight = image.size
  imgData = 1
  extension = "jpg"

  imageAnnotations = annotations.loc[annotations['ImageID'] == imageId]

  CsvRows = []

  for _, row in imageAnnotations.loc[imageAnnotations['LabelName'].isin(classes.keys())].iterrows():

    CsvRow = []

    CsvRow.append(imgHeight)
    CsvRow.append(imgWidth)
    CsvRow.append(imageId)
    CsvRow.append(imageId)
    CsvRow.append(imgData)
    CsvRow.append(extension)
    CsvRow.append(row['XMin'])
    CsvRow.append(row['XMax'])
    CsvRow.append(row['YMin'])
    CsvRow.append(row['YMax'])
    CsvRow.append(row['LabelName'].encode('utf8'))
    CsvRow.append(classes[row['LabelName']])

    CsvRows.append(CsvRow)

  return CsvRows

#### Add `csv` file header

In [80]:
def csvAddHeader(fileName):

  headerRow = []

  headerRow.append('imgHeight')
  headerRow.append('imgWidth')
  headerRow.append('imageId')
  headerRow.append('imageId')
  headerRow.append('imgData')
  headerRow.append('extension')
  headerRow.append('XMin')
  headerRow.append('XMax')
  headerRow.append('YMin')
  headerRow.append('YMax')
  headerRow.append('LabelName')
  headerRow.append('LabelId')

  appendListAsRow(fileName, headerRow)

#### Generate the `tfrecord` files from the provided images & annotations
>The `numShards` variable serves for deciding how many sub-files must be created in order to speed up processing.

In [81]:
def open_sharded_tfrecord_output(exit_stack, base_path, num_shards):
  """Opens all TFRecord shards for writing and adds them to an exit stack.

  Args:
    exit_stack: A context2.ExitStack used to automatically closed the TFRecords
      opened in this function.
    base_path: The base path for all shards
    num_shards: The number of shards

  Returns:
    The list of opened TFRecords. Position k in the list corresponds to shard k.
  """
  tf_record_output_filenames = [
      '{}-{:05d}-of-{:05d}'.format(base_path, idx, num_shards)
      for idx in range(num_shards)
  ]

  tfrecords = [
      exit_stack.enter_context(tf.io.TFRecordWriter(file_name))
      for file_name in tf_record_output_filenames
  ]

  return tfrecords

In [82]:
def open_sharded_csv_output(csvDir, numShards):
  csvFileNames = []

  for i in range(0, numShards):
    csvFileNames.append('{}-{:05d}-of-{:05d}.csv'.format(csvDir, i, numShards))

  return csvFileNames

In [83]:
def tfrecordGenerator(
    classesFile,
    classDescriptionsFile,
    annotationsFile,
    imagesDir,
    outputFile,
    numShards,
    csvNeeded,
    csvDir
):

    classes = list(filter(None, open(classesFile).read().split('\n')))
    classes = {name: idx + 1 for idx, name in enumerate(classes)}
    nameMustPresentClasses = ["Vehicle registration plate"]

    classDescriptions = {row[0]: row[1] for _, row in pd.read_csv(classDescriptionsFile, header=None).iterrows()}

    annotations = pd.read_csv(annotationsFile)
    annotations['ImageID'] = annotations['ImageID'].astype(str)
    annotations['LabelName'] = annotations['LabelName'].map(lambda n: classDescriptions[n])

    #list of labels to rename to Vehicle
    toReplace = ["Car", "Airplane", "Helicopter", "Boat", "Motorcycle", "Bus", "Taxi", "Truck", "Ambulance"]

    #generate list of labels to keep
    toKeep = toReplace.copy()
    for item in nameMustPresentClasses:
      toKeep.append(item)
    #keep only labels in toKeep
    annotations = annotations[annotations['LabelName'].isin(toKeep)]

    #change toReplace labels to Vehicle
    annotations.loc[annotations['LabelName'].isin(toReplace), 'LabelName'] = "Vehicle"

    images = tf.io.gfile.glob(imagesDir + '/*/*.jpg')
    images = map(lambda i: (os.path.basename(i).split('.jpg')[0], i), images)
    images = dict(images)
    print(f'{len(images)} images found')

    with contextlib2.ExitStack() as tfRecordCloseStack:

      outputRecords = open_sharded_tfrecord_output(tfRecordCloseStack, outputFile, numShards)
      outputCsvs = open_sharded_csv_output(csvDir, numShards)

      index = 0

      if(csvNeeded == True):
        for filePath in outputCsvs: 
          csvAddHeader(filePath)

      for imageId, path in images.items():

        imageAnnotations = annotations.loc[annotations['ImageID'] == imageId]
        TFRecord = createRecord(path, imageId, nameMustPresentClasses, classes, imageAnnotations)

        if(TFRecord is None):
          continue

        outputShardIndex = index % numShards
        outputRecords[outputShardIndex].write(TFRecord.SerializeToString())

        if(csvNeeded == True):
          imageRows = createCsvRowsOfImage(path, imageId, classes, annotations)
          for row in imageRows:
            appendListAsRow(outputCsvs[outputShardIndex], row)

        index += 1

    print('TFRecords has been successfully created')

##4. Generate dataset
>The generated `tfrecord` files need to be zipped and saved to the Drive directory.

#### Create output directories

In [84]:
if not os.path.exists(recordsPath):
  os.makedirs(recordsPath)

if not os.path.exists(csvPath):
  os.makedirs(csvPath)

#### Generate train, test, validation `tfrecord` files
>`subsetTypes` contains the subset names, `numShards` contains their corresponding values to be sharded into. More than a few thousand `tfrecord` rows dramatically slow down the process. `numShards=1` means sharding is not needed at all.

In [85]:
# if the data is newly downloaded, use this:
#dataSourcePath = datasetRootPath
# if the data has been extracted, use this:
dataSourcePath = localPath + "/data"

createClassTxt(dataSourcePath, classNames)
createClassPbtxt(recordsPath, classNames)

# train, test, validation subsets
subsetTypes = ['train', 'test', 'validation']
numShards = [10, 3, 1]

csvNeeded = True
csvDirName = ""

for subsetType, numShard in zip(subsetTypes, numShards):

  print(f'***Generating {subsetType} tfrecord files [sharded into {numShard} piece(s)]***')

  classesFile = dataSourcePath + '/classes.txt'
  classDescriptionsFile = dataSourcePath + '/csv_folder/class-descriptions-boxable.csv'
  annotationsFile = dataSourcePath + '/csv_folder/' + subsetType + '-annotations-bbox.csv'
  imagesDir = dataSourcePath + '/Dataset/' + subsetType
  outputFile = recordsPath + '/' + subsetType + 'Dataset.tfrecord'
  csvDirName = csvPath + '/' + subsetType

  tfrecordGenerator(
      classesFile,
      classDescriptionsFile,
      annotationsFile,
      imagesDir,
      outputFile,
      numShard,
      csvNeeded,
      csvDirName
  )

  print('')

Class names txt file creation successful. Destination: /content/data/classes.txt
Class names pbtxt file creation successful. Destination: /content/generated/records/classes.pbtxt
***Generating train tfrecord files [sharded into 10 piece(s)]***
5368 images found
TFRecords has been successfully created

***Generating test tfrecord files [sharded into 3 piece(s)]***
1113 images found
TFRecords has been successfully created

***Generating validation tfrecord files [sharded into 1 piece(s)]***
386 images found
TFRecords has been successfully created



###Zip & save `tfrecord` files to Drive

In [86]:
outputFileName = "datasetRecords"
os.chdir(localPath)
shutil.make_archive(outputFileName, 'zip', recordsPath)
shutil.move(localPath + '/' + outputFileName + '.zip', dataPath)

'/content/drive/My Drive/Machine Learning/Stolen vehicle detection/data/datasetRecords.zip'

###Zip & save `csv` files to Drive

In [87]:
outputFileName = "datasetCsvs"
os.chdir(localPath)
shutil.make_archive(outputFileName, 'zip', csvPath)
shutil.move(localPath + '/' + outputFileName + '.zip', dataPath)

'/content/drive/My Drive/Machine Learning/Stolen vehicle detection/data/datasetCsvs.zip'

### How to use the sharded dataset

In [ ]:
tf_record_input_reader {
  input_path: "/path/to/trainDataset.tfrecord-?????-of-00010"
}

##Force garbage collection

In [ ]:
gc.collect()

759